<a href="https://colab.research.google.com/github/EdnilsonRosa/Data-Science/blob/master/Avalia%C3%A7%C3%A3o%20Big%20Data%202019-05-25%2000%3A56.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Avaliação Big Data - Phyton 2

## Inicializando o ambiente e carregando as bibliotecas

In [202]:
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.4.2/spark-2.4.2-bin-hadoop2.7.tgz
!tar xf spark-2.4.2-bin-hadoop2.7.tgz
!pip install -q findspark

!java -version

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-1.11.0-openjdk-amd64/"
os.environ["SPARK_HOME"] = "/content/spark-2.4.2-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

### Chamada da biblioteca

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.types import DoubleType

### Inicializaçaõ da Sessão do Spark - 

spark = SparkSession.builder \
                    .appName("Tutorial PySpark SQL") \
                    .config("spark.some.config.option", "some-value") \
                    .getOrCreate()

openjdk version "11.0.3" 2019-04-16
OpenJDK Runtime Environment (build 11.0.3+7-Ubuntu-1ubuntu218.04.1)
OpenJDK 64-Bit Server VM (build 11.0.3+7-Ubuntu-1ubuntu218.04.1, mixed mode, sharing)


In [203]:
!pip install jellyfish
import jellyfish

In [0]:
import pandas as pd

## Montando o Drive e carregando o arquivo

In [597]:
### Montagem de drive do Google
### Será solicitada autorização
### O Colab tem como raiz uma pasta 'content'

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
dfa = spark.read.csv('gdrive/My Drive/Colab Notebooks/base_sintetica_ascii_a.csv', header=True, sep=',')

In [599]:
dfa.count()

540

In [0]:
dfb = spark.read.csv('gdrive/My Drive/Colab Notebooks/base_sintetica_ascii_b.csv', header=True, sep=',')

In [601]:
dfb.count()

143

In [602]:
dfa.show(5)

+---+--------------+---------+---------+--------------+----------+--------------------+--------------------+------+-----+--------------------+--------------------+------+---------+--------------+----+
| id|        bairro|      cep|   cidade|           cpf| data_nasc|               email|            endereco|estado|idade|                 mae|                nome|numero|    signo|tipo_sanguineo|_c15|
+---+--------------+---------+---------+--------------+----------+--------------------+--------------------+------+-----+--------------------+--------------------+------+---------+--------------+----+
|  0|   Vila Sarney|65090-677| Sao Luis|097.627.958-49|10/09/1948|vicenteolivermont...|2a Travessa do Ri...|    MA|   71|     Louise Beatriz |Vicente Oliver Mo...|   738|   Virgem|           AB-|null|
|  1|Vila Sao Jorge|75044-220| Anapolis|685.096.335-09|24/04/1995|miguelbernardogon...|  Praca Joao Salomao|    GO|   24|     Raquel Barbara |Miguel Bernardo G...|   805|    Touro|            A-|n

In [603]:
dfb.limit(5).toPandas()

,id,cep,cidade,cpf,data_nasc,endereco,estado,mae,nome,tipo_sanguineo
0,01,69304-350,Boa Vista,506.865.202-46,22/11/1965,Avenida Mario Homem de Melo,RR,Isabelle Marcia Antonella,Felipe Bruno dos Santos,AB-
1,02,78132-360,Varzea Grande,325.828.867-42,None,Rua Rio Cuiaba,MT,Alana Isadora,Marcio Heitor Silveira,AB-
2,03,88810-336,Criciuma,284.536.218-88,23/09/1961,Rua Romeu Lopes de Carvalho,SC,Caroline Gabriela,Theo Mario Victor Barbosa,A-
3,04,60874-405,Fortaleza,390.375.925-20,11/05/1981,None,CE,Tereza Bruna,Pietro Henrique de Paula,AB-
4,05,79106-140,Campo Grande,756.005.387-40,14/12/1941,Rua Pato Branco,MS,Alice Leticia,Kaique Thiago de Paula,A-


In [604]:
dfa.printSchema()
dfb.printSchema()

root
 |-- id: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- cpf: string (nullable = true)
 |-- data_nasc: string (nullable = true)
 |-- email: string (nullable = true)
 |-- endereco: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- idade: string (nullable = true)
 |-- mae: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- signo: string (nullable = true)
 |-- tipo_sanguineo: string (nullable = true)
 |-- _c15: string (nullable = true)

root
 |-- id: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- cpf: string (nullable = true)
 |-- data_nasc: string (nullable = true)
 |-- endereco: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- mae: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- tipo_sanguineo: string (nullable = true)



## Tratamento de Todos os Atributo Missing


In [0]:
# Guardando data frames originais sem pré=processamento
dfa_o = dfa
dfb_o = dfb

In [606]:
conta_null_a = dfa.filter(F.col('id').isNull()).count(), dfa.filter(F.col('bairro').isNull()).count(), dfa.filter(F.col('cep').isNull()).count()
conta_null_a += dfa.filter(F.col('cidade').isNull()).count(), dfa.filter(F.col('cpf').isNull()).count(), dfa.filter(F.col('data_nasc').isNull()).count()
conta_null_a += dfa.filter(F.col('email').isNull()).count(), dfa.filter(F.col('endereco').isNull()).count(), dfa.filter(F.col('estado').isNull()).count()
conta_null_a += dfa.filter(F.col('idade').isNull()).count(), dfa.filter(F.col('mae').isNull()).count(), dfa.filter(F.col('nome').isNull()).count()
conta_null_a += dfa.filter(F.col('numero').isNull()).count(), dfa.filter(F.col('signo').isNull()).count(), dfa.filter(F.col('tipo_sanguineo').isNull()).count()
conta_null_a

(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

In [607]:
conta_null_b = dfb.filter(F.col('id').isNull()).count(), dfb.filter(F.col('cep').isNull()).count()
conta_null_b += dfb.filter(F.col('cidade').isNull()).count(), dfb.filter(F.col('cpf').isNull()).count(), dfb.filter(F.col('data_nasc').isNull()).count()
conta_null_b += dfb.filter(F.col('endereco').isNull()).count(), dfb.filter(F.col('estado').isNull()).count()
conta_null_b += dfb.filter(F.col('mae').isNull()).count(), dfb.filter(F.col('nome').isNull()).count(), dfb.filter(F.col('tipo_sanguineo').isNull()).count()
conta_null_b

(0, 9, 8, 0, 6, 2, 5, 0, 2, 8)

In [0]:
# Trocando campos nulos por espaço em branco (os valores anteriores são sobrescritos ao utilizar o mesmo nome de coluna)
dfb = dfb.withColumn('cep', F.when(F.col('cep').isNull(),"00000-000").otherwise(F.col('cep')))
dfb = dfb.withColumn('cidade', F.when(F.col('cidade').isNull(),"cidade").otherwise(F.col('cidade')))
dfb = dfb.withColumn('data_nasc', F.when(F.col('data_nasc').isNull(),"01/01/1980").otherwise(F.col('data_nasc')))
dfb = dfb.withColumn('endereco', F.when(F.col('endereco').isNull(),"Rua").otherwise(F.col('endereco')))
dfb = dfb.withColumn('estado', F.when(F.col('estado').isNull(),"NA").otherwise(F.col('estado')))
dfb = dfb.withColumn('nome', F.when(F.col('nome').isNull()," ").otherwise(F.col('nome')))
dfb = dfb.withColumn('tipo_sanguineo', F.when(F.col('tipo_sanguineo').isNull()," ").otherwise(F.col('tipo_sanguineo')))

In [609]:
conta_null_b = dfb.filter(F.col('id').isNull()).count(), dfb.filter(F.col('cep').isNull()).count()
conta_null_b += dfb.filter(F.col('cidade').isNull()).count(), dfb.filter(F.col('cpf').isNull()).count(), dfb.filter(F.col('data_nasc').isNull()).count()
conta_null_b += dfb.filter(F.col('endereco').isNull()).count(), dfb.filter(F.col('estado').isNull()).count()
conta_null_b += dfb.filter(F.col('mae').isNull()).count(), dfb.filter(F.col('nome').isNull()).count(), dfb.filter(F.col('tipo_sanguineo').isNull()).count()
conta_null_b

(0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

In [610]:
dfb.limit(5).toPandas()

,id,cep,cidade,cpf,data_nasc,endereco,estado,mae,nome,tipo_sanguineo
0,01,69304-350,Boa Vista,506.865.202-46,22/11/1965,Avenida Mario Homem de Melo,RR,Isabelle Marcia Antonella,Felipe Bruno dos Santos,AB-
1,02,78132-360,Varzea Grande,325.828.867-42,01/01/1980,Rua Rio Cuiaba,MT,Alana Isadora,Marcio Heitor Silveira,AB-
2,03,88810-336,Criciuma,284.536.218-88,23/09/1961,Rua Romeu Lopes de Carvalho,SC,Caroline Gabriela,Theo Mario Victor Barbosa,A-
3,04,60874-405,Fortaleza,390.375.925-20,11/05/1981,Rua,CE,Tereza Bruna,Pietro Henrique de Paula,AB-
4,05,79106-140,Campo Grande,756.005.387-40,14/12/1941,Rua Pato Branco,MS,Alice Leticia,Kaique Thiago de Paula,A-


## **Outros Tratamentos**

In [0]:
# Eliminação de possíveis espaços duplos nos nomes
dfa = dfa.withColumn("nome", F.regexp_replace(F.col("nome"), '\s+', ' '))
dfb = dfb.withColumn("nome", F.regexp_replace(F.col("nome"), '\s+', ' '))
dfa = dfa.withColumn("mae", F.regexp_replace(F.col("mae"), '\s+', ' '))
dfb = dfb.withColumn("mae", F.regexp_replace(F.col("mae"), '\s+', ' '))
dfa = dfa.withColumn("cidade", F.regexp_replace(F.col("cidade"), '\s+', ' '))
dfb = dfb.withColumn("cidade", F.regexp_replace(F.col("cidade"), '\s+', ' '))
dfa = dfa.withColumn("endereco", F.regexp_replace(F.col("endereco"), '\s+', ' '))
dfb = dfb.withColumn("endereco", F.regexp_replace(F.col("endereco"), '\s+', ' '))

# Substituição de possiveis caracteres especiais por espaço em branco
dfa = dfa.withColumn("nome", F.regexp_replace(F.col("nome"), r'[^a-zA-Z ]+', ' '))
dfb = dfb.withColumn("nome", F.regexp_replace(F.col("nome"), r'[^a-zA-Z ]+', ' '))
dfa = dfa.withColumn("mae", F.regexp_replace(F.col("mae"), r'[^a-zA-Z ]+', ' '))
dfb = dfb.withColumn("mae", F.regexp_replace(F.col("mae"), r'[^a-zA-Z ]+', ' '))
dfa = dfa.withColumn("cidade", F.regexp_replace(F.col("cidade"), r'[^a-zA-Z ]+', ' '))
dfb = dfb.withColumn("cidade", F.regexp_replace(F.col("cidade"), r'[^a-zA-Z ]+', ' '))
dfa = dfa.withColumn("endereco", F.regexp_replace(F.col("endereco"), r'[^a-zA-Z ]+', ' '))


## Renomeando colunas para o linkage (necessita nomes diferentes)

In [0]:
dfb = dfb.withColumnRenamed('id','id_b')
dfb = dfb.withColumnRenamed('cep','cep_b')
dfb = dfb.withColumnRenamed('cidade','cidade_b')
dfb = dfb.withColumnRenamed('cpf','cpf_b')
dfb = dfb.withColumnRenamed('data_nasc','data_nasc_b')
dfb = dfb.withColumnRenamed('endereco','endereco_b')
dfb = dfb.withColumnRenamed('estado','estado_b')
dfb = dfb.withColumnRenamed('mae','mae_b')
dfb = dfb.withColumnRenamed('nome','nome_b')
dfb = dfb.withColumnRenamed('tipo_sanguineo','tipo_sanguineo_b')

In [613]:
dfa.printSchema()
dfb.printSchema()

root
 |-- id: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- cpf: string (nullable = true)
 |-- data_nasc: string (nullable = true)
 |-- email: string (nullable = true)
 |-- endereco: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- idade: string (nullable = true)
 |-- mae: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- signo: string (nullable = true)
 |-- tipo_sanguineo: string (nullable = true)
 |-- _c15: string (nullable = true)

root
 |-- id_b: string (nullable = true)
 |-- cep_b: string (nullable = true)
 |-- cidade_b: string (nullable = true)
 |-- cpf_b: string (nullable = true)
 |-- data_nasc_b: string (nullable = true)
 |-- endereco_b: string (nullable = true)
 |-- estado_b: string (nullable = true)
 |-- mae_b: string (nullable = true)
 |-- nome_b: string (nullable = true)
 |-- tipo_sanguineo_b: string (nullabl

## Fazendo Linkage

In [0]:
# CrossJoin somente com colunas que serão utilizadas na análise

dfl = dfa.select('id','cep','cidade','data_nasc','endereco','estado','idade','mae','nome','tipo_sanguineo','cpf').crossJoin( \
      dfb.select('id_b','cep_b','cidade_b','data_nasc_b','endereco_b','estado_b','mae_b','nome_b','tipo_sanguineo_b','cpf_b'))

In [615]:
dfl.limit(5).show()

+---+---------+--------+----------+--------------------+------+-----+---------------+--------------------+--------------+--------------+----+---------+-------------+-----------+--------------------+--------+--------------------+--------------------+----------------+--------------+
| id|      cep|  cidade| data_nasc|            endereco|estado|idade|            mae|                nome|tipo_sanguineo|           cpf|id_b|    cep_b|     cidade_b|data_nasc_b|          endereco_b|estado_b|               mae_b|              nome_b|tipo_sanguineo_b|         cpf_b|
+---+---------+--------+----------+--------------------+------+-----+---------------+--------------------+--------------+--------------+----+---------+-------------+-----------+--------------------+--------+--------------------+--------------------+----------------+--------------+
|  0|65090-677|Sao Luis|10/09/1948| a Travessa do Ri...|    MA|   71|Louise Beatriz |Vicente Oliver Mo...|           AB-|097.627.958-49|  01|69304-350|   

## Criação de Funções para extração do primeiro e último nome

In [0]:
def criaPrimeiroNome(col):
  return col.split(' ') [0]
udf_criaPrimeiroNome = F.udf(criaPrimeiroNome, StringType())

In [0]:
def criaUltimoNome(col):
  return col.split(' ') [-1]
udf_criaUltimoNome = F.udf(criaUltimoNome, StringType())

## Criação das Colunas dos Primeiros e Ultimos Nomes

In [0]:
dfl = dfl.withColumn('primeiro_nome_a', udf_criaPrimeiroNome(F.col('nome')))
dfl = dfl.withColumn('primeiro_nome_b', udf_criaPrimeiroNome(F.col('nome_b')))
dfl = dfl.withColumn('ultimo_nome_a', udf_criaUltimoNome(F.col('nome')))
dfl = dfl.withColumn('ultimo_nome_b', udf_criaUltimoNome(F.col('nome_b')))

In [619]:
dfl.printSchema()

root
 |-- id: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- data_nasc: string (nullable = true)
 |-- endereco: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- idade: string (nullable = true)
 |-- mae: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- tipo_sanguineo: string (nullable = true)
 |-- cpf: string (nullable = true)
 |-- id_b: string (nullable = true)
 |-- cep_b: string (nullable = true)
 |-- cidade_b: string (nullable = true)
 |-- data_nasc_b: string (nullable = true)
 |-- endereco_b: string (nullable = true)
 |-- estado_b: string (nullable = true)
 |-- mae_b: string (nullable = true)
 |-- nome_b: string (nullable = true)
 |-- tipo_sanguineo_b: string (nullable = true)
 |-- cpf_b: string (nullable = true)
 |-- primeiro_nome_a: string (nullable = true)
 |-- primeiro_nome_b: string (nullable = true)
 |-- ultimo_nome_a: string (nullable = true)
 |-- ultimo_nome_b: string (nullable = true

In [620]:
dfl.show(5)

+---+---------+--------+----------+--------------------+------+-----+---------------+--------------------+--------------+--------------+----+---------+-------------+-----------+--------------------+--------+--------------------+--------------------+----------------+--------------+---------------+---------------+-------------+-------------+
| id|      cep|  cidade| data_nasc|            endereco|estado|idade|            mae|                nome|tipo_sanguineo|           cpf|id_b|    cep_b|     cidade_b|data_nasc_b|          endereco_b|estado_b|               mae_b|              nome_b|tipo_sanguineo_b|         cpf_b|primeiro_nome_a|primeiro_nome_b|ultimo_nome_a|ultimo_nome_b|
+---+---------+--------+----------+--------------------+------+-----+---------------+--------------------+--------------+--------------+----+---------+-------------+-----------+--------------------+--------+--------------------+--------------------+----------------+--------------+---------------+---------------+---

## Criação das colunas com nomes fonéticos

In [0]:
def criaMetaphone(col):
  return jellyfish.metaphone(col)
udf_criaMetaphone = F.udf(criaMetaphone, StringType())

In [0]:
dfl = dfl.withColumn('primeiro_nome_fonetico_a', udf_criaMetaphone(F.col('primeiro_nome_a')))
dfl = dfl.withColumn('primeiro_nome_fonetico_b', udf_criaMetaphone(F.col('primeiro_nome_b')))
dfl = dfl.withColumn('ultimo_nome_fonetico_a', udf_criaMetaphone(F.col('ultimo_nome_a')))
dfl = dfl.withColumn('ultimo_nome_fonetico_b', udf_criaMetaphone(F.col('ultimo_nome_b')))

In [0]:
dfl = dfl.withColumn('nome_fonetico_a', udf_criaMetaphone(F.col('nome')))
dfl = dfl.withColumn('nome_fonetico_b', udf_criaMetaphone(F.col('nome_b')))
dfl = dfl.withColumn('nome_fonetico_mae_a', udf_criaMetaphone(F.col('mae')))
dfl = dfl.withColumn('nome_fonetico_mae_b', udf_criaMetaphone(F.col('mae_b')))

In [624]:
dfl.show(5)

+---+---------+--------+----------+--------------------+------+-----+---------------+--------------------+--------------+--------------+----+---------+-------------+-----------+--------------------+--------+--------------------+--------------------+----------------+--------------+---------------+---------------+-------------+-------------+------------------------+------------------------+----------------------+----------------------+---------------+---------------+-------------------+-------------------+
| id|      cep|  cidade| data_nasc|            endereco|estado|idade|            mae|                nome|tipo_sanguineo|           cpf|id_b|    cep_b|     cidade_b|data_nasc_b|          endereco_b|estado_b|               mae_b|              nome_b|tipo_sanguineo_b|         cpf_b|primeiro_nome_a|primeiro_nome_b|ultimo_nome_a|ultimo_nome_b|primeiro_nome_fonetico_a|primeiro_nome_fonetico_b|ultimo_nome_fonetico_a|ultimo_nome_fonetico_b|nome_fonetico_a|nome_fonetico_b|nome_fonetico_mae_a|n

In [625]:
dfl.printSchema()

root
 |-- id: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- data_nasc: string (nullable = true)
 |-- endereco: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- idade: string (nullable = true)
 |-- mae: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- tipo_sanguineo: string (nullable = true)
 |-- cpf: string (nullable = true)
 |-- id_b: string (nullable = true)
 |-- cep_b: string (nullable = true)
 |-- cidade_b: string (nullable = true)
 |-- data_nasc_b: string (nullable = true)
 |-- endereco_b: string (nullable = true)
 |-- estado_b: string (nullable = true)
 |-- mae_b: string (nullable = true)
 |-- nome_b: string (nullable = true)
 |-- tipo_sanguineo_b: string (nullable = true)
 |-- cpf_b: string (nullable = true)
 |-- primeiro_nome_a: string (nullable = true)
 |-- primeiro_nome_b: string (nullable = true)
 |-- ultimo_nome_a: string (nullable = true)
 |-- ultimo_nome_b: string (nullable = true

## 1 - Comparação Fonética pelo Nome e Nome da Mãe com Jaro Winkler


In [0]:
#Criação da Função de Similaridade Fonética pelo Nome e Nome da Mãe com Jaro Winkler
def compare(primeiro_nome_fonetico_a, ultimo_nome_fonetico_a, nome_fonetico_mae_a, primeiro_nome_fonetico_b, ultimo_nome_fonetico_b, nome_fonetico_mae_b):
  sim = 0
  
  #comparando atributos nominais
  sim = jellyfish.jaro_winkler(primeiro_nome_fonetico_a, primeiro_nome_fonetico_b)
  sim += jellyfish.jaro_winkler(ultimo_nome_fonetico_a, ultimo_nome_fonetico_b)
  sim += jellyfish.jaro_winkler(nome_fonetico_mae_a, nome_fonetico_mae_b)
  
  #Media aritmética simples
  sim = abs(float(sim/3))
  
  return sim

udf_compare = F.udf(compare, StringType())  

In [0]:
dfr = dfl.withColumn('similaridade', 
                     udf_compare(F.col('primeiro_nome_fonetico_a'),
                                 F.col('ultimo_nome_fonetico_a'),
                                 F.col('nome_fonetico_mae_a'),
                                 F.col('primeiro_nome_fonetico_b'),
                                 F.col('ultimo_nome_fonetico_b'),
                                 F.col('nome_fonetico_mae_b')))

In [628]:
dfr.printSchema()

root
 |-- id: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- data_nasc: string (nullable = true)
 |-- endereco: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- idade: string (nullable = true)
 |-- mae: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- tipo_sanguineo: string (nullable = true)
 |-- cpf: string (nullable = true)
 |-- id_b: string (nullable = true)
 |-- cep_b: string (nullable = true)
 |-- cidade_b: string (nullable = true)
 |-- data_nasc_b: string (nullable = true)
 |-- endereco_b: string (nullable = true)
 |-- estado_b: string (nullable = true)
 |-- mae_b: string (nullable = true)
 |-- nome_b: string (nullable = true)
 |-- tipo_sanguineo_b: string (nullable = true)
 |-- cpf_b: string (nullable = true)
 |-- primeiro_nome_a: string (nullable = true)
 |-- primeiro_nome_b: string (nullable = true)
 |-- ultimo_nome_a: string (nullable = true)
 |-- ultimo_nome_b: string (nullable = true

In [629]:
# Exibindo base de resultado
dfr.select(['id', 'id_b', 'nome', 'nome_b', 'cpf', 'cpf_b', 'similaridade']).show(10)

+---+----+--------------------+--------------------+--------------+--------------+-------------------+
| id|id_b|                nome|              nome_b|           cpf|         cpf_b|       similaridade|
+---+----+--------------------+--------------------+--------------+--------------+-------------------+
|  0|  01|Vicente Oliver Mo...|Felipe Bruno dos ...|097.627.958-49|506.865.202-46|                0.6|
|  0|  02|Vicente Oliver Mo...|Marcio Heitor Sil...|097.627.958-49|325.828.867-42| 0.5703703703703703|
|  0|  03|Vicente Oliver Mo...|Theo Mario Victor...|097.627.958-49|284.536.218-88|0.19166666666666668|
|  0|  04|Vicente Oliver Mo...|Pietro Henrique d...|097.627.958-49|390.375.925-20|               0.25|
|  0|  05|Vicente Oliver Mo...|Kaique Thiago de ...|097.627.958-49|756.005.387-40|               0.25|
|  0|  06|Vicente Oliver Mo...|Gabriel Erick Oli...|097.627.958-49|551.341.139-39|0.20502645502645503|
|  0|  07|Vicente Oliver Mo...|Luan Caue Anthony...|097.627.958-49|756.04

In [0]:
# Deduplicando a base (eliminando linhas duplicadas, ficando somente com a de maior similaridade)
dfc = dfr.orderBy(['similaridade'], ascending=False).dropDuplicates(['id_b']).orderBy(['id_b']).select(['id', 'id_b', 'nome','mae', 'nome_b', 'mae_b', 'cpf','cpf_b', 'similaridade'])


In [0]:
# Criação de coluna de comparação de CPFs
dfc = dfc.withColumn('CPFs_Iguais',F.col('cpf') == F.col('cpf_b'))

In [632]:
dfc.describe('similaridade').show()

+-------+-------------------+
|summary|       similaridade|
+-------+-------------------+
|  count|                143|
|   mean| 0.9736581263853991|
| stddev|0.08772987251916958|
|    min| 0.3333333333333333|
|    max|                1.0|
+-------+-------------------+



In [633]:
dfc.count()

143

In [634]:
dfc.filter(dfc['similaridade'] >= '0.9').count()

137

In [635]:
# Exibindo os resultados com similariade maior do que 0,9
dfc.filter(dfc['similaridade'] >= '0.9').orderBy(['similaridade'], ascending=False).show(150)

+---+----+--------------------+--------------------+--------------------+--------------------+--------------+--------------+------------------+-----------+
| id|id_b|                nome|                 mae|              nome_b|               mae_b|           cpf|         cpf_b|      similaridade|CPFs_Iguais|
+---+----+--------------------+--------------------+--------------------+--------------------+--------------+--------------+------------------+-----------+
| 20|  07|Luan Caue Anthony...|Vanessa Camila Ni...|Luan Caue Anthony...|Vanessa Camila Ni...|756.048.583-90|756.048.583-90|               1.0|       true|
| 63|  51|Alexandre Bruno M...|   Isabelly Kamilly |Alexandre Bruno M...|   Isabelly Kamilly |869.696.945-69|869.696.945-69|               1.0|       true|
|126| 138|Heitor Leonardo C...|Regina Gabriela A...|Heitor Leonardo C...|Regina Gabriela A...|870.599.445-40|046.391.337-07|               1.0|      false|
|513|  30|Miguel Francisco ...|    Fernanda Mariah |Miguel Franc

## 2 - Comparação Não Fonética pelo Nome e Nome da Mãe com Jaro Winkler


In [0]:
#Criação da Função de Similaridade
def compare_2(primeiro_nome_a, ultimo_nome_a, mae, primeiro_nome_b, ultimo_nome_b, mae_b):
  sim = 0
  
  #comparando atributos nominais
  sim = jellyfish.jaro_winkler(primeiro_nome_a, primeiro_nome_b)
  sim += jellyfish.jaro_winkler(ultimo_nome_a, ultimo_nome_b)
  sim += jellyfish.jaro_winkler(mae, mae_b)
  
  #Media aritmética simples
  sim = abs(float(sim/3))
  
  return sim

udf_compare_2 = F.udf(compare_2, StringType())  

In [0]:
dfr_2 = dfl.withColumn('similaridade', 
                     udf_compare_2 (F.col('primeiro_nome_a'),
                                    F.col('ultimo_nome_a'),
                                    F.col('mae'),
                                    F.col('primeiro_nome_b'),
                                    F.col('ultimo_nome_b'),
                                    F.col('mae_b')))

In [639]:
# Exibindo base de resultado
dfr_2.select(['id', 'id_b', 'nome', 'nome_b', 'cpf', 'cpf_b', 'similaridade']).show(10)

+---+----+--------------------+--------------------+--------------+--------------+-------------------+
| id|id_b|                nome|              nome_b|           cpf|         cpf_b|       similaridade|
+---+----+--------------------+--------------------+--------------+--------------+-------------------+
|  0|  01|Vicente Oliver Mo...|Felipe Bruno dos ...|097.627.958-49|506.865.202-46|  0.515401573734907|
|  0|  02|Vicente Oliver Mo...|Marcio Heitor Sil...|097.627.958-49|325.828.867-42|0.48709808709808705|
|  0|  03|Vicente Oliver Mo...|Theo Mario Victor...|097.627.958-49|284.536.218-88| 0.4795855379188712|
|  0|  04|Vicente Oliver Mo...|Pietro Henrique d...|097.627.958-49|390.375.925-20|0.40533170533170537|
|  0|  05|Vicente Oliver Mo...|Kaique Thiago de ...|097.627.958-49|756.005.387-40|  0.392063492063492|
|  0|  06|Vicente Oliver Mo...|Gabriel Erick Oli...|097.627.958-49|551.341.139-39|0.31020818955601565|
|  0|  07|Vicente Oliver Mo...|Luan Caue Anthony...|097.627.958-49|756.04

In [0]:
# Deduplicando a base (eliminando linhas duplicadas, ficando somente com a de maior similaridade)
dfc_2 = dfr_2.orderBy(['similaridade'], ascending=False).dropDuplicates(['id_b']).orderBy(['id_b']).select(['id', 'id_b', 'nome','nome_b','cpf','cpf_b', 'similaridade'])


In [0]:
# Criação de coluna de comparação de CPFs
dfc_2 = dfc_2.withColumn('CPFs_Iguais',F.col('cpf') == F.col('cpf_b'))

In [642]:
dfc_2.describe('similaridade').show()

+-------+-------------------+
|summary|       similaridade|
+-------+-------------------+
|  count|                143|
|   mean| 0.9704922334042875|
| stddev|0.09090226746595302|
|    min| 0.3333333333333333|
|    max|                1.0|
+-------+-------------------+



In [643]:
dfc_2.count()

143

In [644]:
dfc_2.filter(dfc_2['similaridade'] >= '0.9').count()

131

In [645]:
# Exibindo os resultados com similariade maior do que 0.9
dfc_2.filter(dfc_2['similaridade'] >= '0.9').orderBy(['similaridade'], ascending=False).show(150)

+---+----+--------------------+--------------------+--------------+--------------+------------------+-----------+
| id|id_b|                nome|              nome_b|           cpf|         cpf_b|      similaridade|CPFs_Iguais|
+---+----+--------------------+--------------------+--------------+--------------+------------------+-----------+
| 20|  07|Luan Caue Anthony...|Luan Caue Anthony...|756.048.583-90|756.048.583-90|               1.0|       true|
|101| 113|Claudio Murilo Au...|        Claudio Rosa|311.265.388-25|748.193.267-08|               1.0|      false|
| 14|  01|Felipe Bruno dos ...|Felipe Bruno dos ...|506.865.202-46|506.865.202-46|               1.0|       true|
| 28|  15|Leandro Cesar Jor...|Leandro Cesar Jor...|181.410.022-95|181.410.022-95|               1.0|       true|
| 35|  22|Mario Kevin Baptista|Mario Kevin Baptista|097.294.987-92|097.294.987-92|               1.0|       true|
|458|  69|Anderson Gustavo ...|Anderson Gustavo ...|081.783.955-09|081.783.955-09|      

## 3 - Comparação Não Fonética por Outros Campos com Jaro Winkler

In [683]:
dfl.printSchema()

root
 |-- id: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- data_nasc: string (nullable = true)
 |-- endereco: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- idade: string (nullable = true)
 |-- mae: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- tipo_sanguineo: string (nullable = true)
 |-- cpf: string (nullable = true)
 |-- id_b: string (nullable = true)
 |-- cep_b: string (nullable = true)
 |-- cidade_b: string (nullable = true)
 |-- data_nasc_b: string (nullable = true)
 |-- endereco_b: string (nullable = true)
 |-- estado_b: string (nullable = true)
 |-- mae_b: string (nullable = true)
 |-- nome_b: string (nullable = true)
 |-- tipo_sanguineo_b: string (nullable = true)
 |-- cpf_b: string (nullable = true)
 |-- primeiro_nome_a: string (nullable = true)
 |-- primeiro_nome_b: string (nullable = true)
 |-- ultimo_nome_a: string (nullable = true)
 |-- ultimo_nome_b: string (nullable = true

In [0]:
def compare_3(nome, mae, data_nasc, tipo_sanguineo, nome_b, mae_b, data_nasc_b, tipo_sanguineo_b):
  sim = 0
  
  #comparando atributos nominais
  sim = jellyfish.jaro_winkler(nome, nome_b)
  sim += jellyfish.jaro_winkler(mae, mae_b)
  sim += jellyfish.jaro_winkler(data_nasc, data_nasc_b)
  sim += jellyfish.jaro_winkler(tipo_sanguineo, tipo_sanguineo_b)
  
  #Media aritmética simples
  sim = abs(float(sim/4))
  
  return sim

udf_compare_3 = F.udf(compare_3, StringType())   

In [0]:
dfr_3 = dfl.withColumn('similaridade', 
                     udf_compare_3 (F.col('nome'),
                                    F.col('mae'),
                                    F.col('data_nasc'),
                                    F.col('tipo_sanguineo'),
                                    F.col('nome_b'),
                                    F.col('mae_b'),
                                    F.col('data_nasc_b'),
                                    F.col('tipo_sanguineo_b')))

In [686]:
# Exibindo base de resultado
dfr_3.select(['nome', 'nome_b', 'mae', 'mae_b', 'data_nasc', 'data_nasc_b', 'tipo_sanguineo', 'tipo_sanguineo_b', 'cpf', 'cpf_b', 'similaridade']).show(10)

+--------------------+--------------------+---------------+--------------------+----------+-----------+--------------+----------------+--------------+--------------+-------------------+
|                nome|              nome_b|            mae|               mae_b| data_nasc|data_nasc_b|tipo_sanguineo|tipo_sanguineo_b|           cpf|         cpf_b|       similaridade|
+--------------------+--------------------+---------------+--------------------+----------+-----------+--------------+----------------+--------------+--------------+-------------------+
|Vicente Oliver Mo...|Felipe Bruno dos ...|Louise Beatriz |Isabelle Marcia A...|10/09/1948| 22/11/1965|           AB-|             AB-|097.627.958-49|506.865.202-46|  0.644043416326025|
|Vicente Oliver Mo...|Marcio Heitor Sil...|Louise Beatriz |       Alana Isadora|10/09/1948| 01/01/1980|           AB-|             AB-|097.627.958-49|325.828.867-42| 0.7075705695270912|
|Vicente Oliver Mo...|Theo Mario Victor...|Louise Beatriz |  Caroline 

In [0]:
# Deduplicando a base (eliminando linhas duplicadas, ficando somente com a de maior similaridade)
dfc_3 = dfr_3.orderBy(['similaridade'], ascending=False).dropDuplicates(['id_b']).orderBy(['id_b']).select(['id', 'id_b', 'nome','nome_b', 'mae', 'mae_b', 'cpf','cpf_b', 'similaridade'])


In [0]:
# Criação de coluna de comparação de CPFs
dfc_3 = dfc_3.withColumn('CPFs_Iguais',F.col('cpf') == F.col('cpf_b'))

In [689]:
dfc_3.describe('similaridade').show()

+-------+-------------------+
|summary|       similaridade|
+-------+-------------------+
|  count|                143|
|   mean| 0.9118457754736309|
| stddev|0.10153252972550626|
|    min| 0.7166666666666667|
|    max|                1.0|
+-------+-------------------+



In [690]:
dfc_3.count()

143

In [691]:
dfc_3.filter(dfc_3['similaridade'] >= '0.9').count()

88

In [692]:
# Exibindo os resultados com similariade maior do que 0.9
dfc_3.filter(dfc_3['similaridade'] >= '0.9').orderBy(['similaridade'], ascending=False).show(150)

+---+----+--------------------+--------------------+--------------------+--------------------+--------------+--------------+------------------+-----------+
| id|id_b|                nome|              nome_b|                 mae|               mae_b|           cpf|         cpf_b|      similaridade|CPFs_Iguais|
+---+----+--------------------+--------------------+--------------------+--------------------+--------------+--------------+------------------+-----------+
|512|  29|Calebe Osvaldo Ba...|Calebe Osvaldo Ba...|Stella Raimunda S...|Stella Raimunda S...|797.175.014-59|797.175.014-59|               1.0|       true|
| 63|  51|Alexandre Bruno M...|Alexandre Bruno M...|   Isabelly Kamilly |   Isabelly Kamilly |869.696.945-69|869.696.945-69|               1.0|       true|
|460|  71|Augusto Bruno da ...|Augusto Bruno da ...|        Aurora Luna |        Aurora Luna |302.706.239-57|302.706.239-57|               1.0|       true|
| 28|  15|Leandro Cesar Jor...|Leandro Cesar Jor...|Jessica Jaqu

## 4 - Comparação Fonética por Outros Campos com Jaro Winkler

In [658]:
dfl.printSchema()

root
 |-- id: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- data_nasc: string (nullable = true)
 |-- endereco: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- idade: string (nullable = true)
 |-- mae: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- tipo_sanguineo: string (nullable = true)
 |-- cpf: string (nullable = true)
 |-- id_b: string (nullable = true)
 |-- cep_b: string (nullable = true)
 |-- cidade_b: string (nullable = true)
 |-- data_nasc_b: string (nullable = true)
 |-- endereco_b: string (nullable = true)
 |-- estado_b: string (nullable = true)
 |-- mae_b: string (nullable = true)
 |-- nome_b: string (nullable = true)
 |-- tipo_sanguineo_b: string (nullable = true)
 |-- cpf_b: string (nullable = true)
 |-- primeiro_nome_a: string (nullable = true)
 |-- primeiro_nome_b: string (nullable = true)
 |-- ultimo_nome_a: string (nullable = true)
 |-- ultimo_nome_b: string (nullable = true

In [0]:
def compare_4(nome_fonetico_a, nome_fonetico_mae_a, data_nasc, tipo_sanguineo, nome_fonetico_b, nome_fonetico_mae_b, data_nasc_b, tipo_sanguineo_b):
  sim = 0
  
  #comparando atributos nominais
  sim = jellyfish.jaro_winkler(nome_fonetico_a, nome_fonetico_b)
  sim += jellyfish.jaro_winkler(nome_fonetico_mae_a, nome_fonetico_mae_b)
  sim += jellyfish.jaro_winkler(data_nasc, data_nasc_b)
  sim += jellyfish.jaro_winkler(tipo_sanguineo, tipo_sanguineo_b)
  
  #Media aritmética simples
  sim = abs(float(sim/4))
  
  return sim

udf_compare_4 = F.udf(compare_4, StringType())   

In [0]:
dfr_4 = dfl.withColumn('similaridade', 
                     udf_compare_4 (F.col('nome_fonetico_a'),
                                    F.col('nome_fonetico_mae_a'),
                                    F.col('data_nasc'),
                                    F.col('tipo_sanguineo'),
                                    F.col('nome_fonetico_b'),
                                    F.col('nome_fonetico_mae_b'),
                                    F.col('data_nasc_b'),
                                    F.col('tipo_sanguineo_b')))

In [661]:
# Exibindo base de resultado
dfr_4.select(['nome', 'nome_b', 'mae', 'mae_b', 'data_nasc', 'data_nasc_b', 'tipo_sanguineo', 'tipo_sanguineo_b', 'cpf', 'cpf_b', 'similaridade']).show(10)

+--------------------+--------------------+---------------+--------------------+----------+-----------+--------------+----------------+--------------+--------------+-------------------+
|                nome|              nome_b|            mae|               mae_b| data_nasc|data_nasc_b|tipo_sanguineo|tipo_sanguineo_b|           cpf|         cpf_b|       similaridade|
+--------------------+--------------------+---------------+--------------------+----------+-----------+--------------+----------------+--------------+--------------+-------------------+
|Vicente Oliver Mo...|Felipe Bruno dos ...|Louise Beatriz |Isabelle Marcia A...|10/09/1948| 22/11/1965|           AB-|             AB-|097.627.958-49|506.865.202-46| 0.6938492063492063|
|Vicente Oliver Mo...|Marcio Heitor Sil...|Louise Beatriz |       Alana Isadora|10/09/1948| 01/01/1980|           AB-|             AB-|097.627.958-49|325.828.867-42| 0.7719246031746031|
|Vicente Oliver Mo...|Theo Mario Victor...|Louise Beatriz |  Caroline 

In [0]:
# Deduplicando a base (eliminando linhas duplicadas, ficando somente com a de maior similaridade)
dfc_4 = dfr_4.orderBy(['similaridade'], ascending=False).dropDuplicates(['id_b']).orderBy(['id_b']).select(['id', 'id_b', 'nome', 'nome_b', 'mae', 'mae_b', 'cpf','cpf_b', 'similaridade'])


In [0]:
# Criação de coluna de comparação de CPFs
dfc_4 = dfc_4.withColumn('CPFs_Iguais',F.col('cpf') == F.col('cpf_b'))

In [664]:
dfc_4.printSchema()

root
 |-- id: string (nullable = true)
 |-- id_b: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- nome_b: string (nullable = true)
 |-- mae: string (nullable = true)
 |-- mae_b: string (nullable = true)
 |-- cpf: string (nullable = true)
 |-- cpf_b: string (nullable = true)
 |-- similaridade: string (nullable = true)
 |-- CPFs_Iguais: boolean (nullable = true)



In [0]:
#dfc_4 = dfc_4.withColumn("similaridade", dfc_4["similaridade"].cast(DoubleType()).alias("similaridade"))

In [666]:
dfc_4.describe('similaridade').show()

+-------+-------------------+
|summary|       similaridade|
+-------+-------------------+
|  count|                143|
|   mean| 0.9160287712357603|
| stddev|0.09672434172823503|
|    min| 0.7166666666666667|
|    max|                1.0|
+-------+-------------------+



In [667]:
dfc_4.count()

143

In [668]:
dfc_4.filter(dfc_4['similaridade'] >= '0.9').count()

95

In [682]:
dfc_4.filter(dfc_4['similaridade'] >= '0.9').orderBy(['similaridade'], ascending=False).show(150)

+---+----+--------------------+--------------------+--------------------+--------------------+--------------+--------------+------------------+-----------+
| id|id_b|                nome|              nome_b|                 mae|               mae_b|           cpf|         cpf_b|      similaridade|CPFs_Iguais|
+---+----+--------------------+--------------------+--------------------+--------------------+--------------+--------------+------------------+-----------+
| 14|  01|Felipe Bruno dos ...|Felipe Bruno dos ...|Isabelle Marcia A...|Isabelle Marcia A...|506.865.202-46|506.865.202-46|               1.0|       true|
|474|  85|Renan Bernardo da...|Renan Bernardo da...|Mariana Laura Isa...|Mariana Laura Isa...|725.521.191-74|725.521.191-74|               1.0|       true|
|458|  69|Anderson Gustavo ...|Anderson Gustavo ...|Vitoria Catarina ...|Vitoria Catarina ...|081.783.955-09|081.783.955-09|               1.0|       true|
|513|  30|Miguel Francisco ...|Miguel Francisco ...|    Fernanda

## 5  - Comparação por Outros Campos com Hamming Distance

In [671]:
dfl.printSchema()

root
 |-- id: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- data_nasc: string (nullable = true)
 |-- endereco: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- idade: string (nullable = true)
 |-- mae: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- tipo_sanguineo: string (nullable = true)
 |-- cpf: string (nullable = true)
 |-- id_b: string (nullable = true)
 |-- cep_b: string (nullable = true)
 |-- cidade_b: string (nullable = true)
 |-- data_nasc_b: string (nullable = true)
 |-- endereco_b: string (nullable = true)
 |-- estado_b: string (nullable = true)
 |-- mae_b: string (nullable = true)
 |-- nome_b: string (nullable = true)
 |-- tipo_sanguineo_b: string (nullable = true)
 |-- cpf_b: string (nullable = true)
 |-- primeiro_nome_a: string (nullable = true)
 |-- primeiro_nome_b: string (nullable = true)
 |-- ultimo_nome_a: string (nullable = true)
 |-- ultimo_nome_b: string (nullable = true

In [0]:
def compare_5(nome, mae, data_nasc, tipo_sanguineo, nome_b, mae_b, data_nasc_b, tipo_sanguineo_b):
  sim = 0
  
  #comparando atributos nominais
  sim = 1 - jellyfish.hamming_distance(nome, nome_b)
  sim += 1 - jellyfish.hamming_distance(mae, mae_b)
  sim += 1 - jellyfish.hamming_distance(data_nasc, data_nasc_b)
  sim += 1 - jellyfish.hamming_distance(tipo_sanguineo, tipo_sanguineo_b)
  
  #Media aritmética simples
  sim = abs(float(sim/4))
  
  return sim

udf_compare_5 = F.udf(compare_5, StringType())   

In [0]:
dfr_5 = dfl.withColumn('distancia', 
                     udf_compare_5 (F.col('nome'),
                                    F.col('mae'),
                                    F.col('data_nasc'),
                                    F.col('tipo_sanguineo'),
                                    F.col('nome_b'),
                                    F.col('mae_b'),
                                    F.col('data_nasc_b'),
                                    F.col('tipo_sanguineo_b')))

In [674]:
# Exibindo base de resultado
dfr_5.select(['nome', 'nome_b', 'mae', 'mae_b', 'data_nasc', 'data_nasc_b', 'tipo_sanguineo', 'tipo_sanguineo_b', 'cpf', 'cpf_b', 'distancia']).show(10)

+--------------------+--------------------+---------------+--------------------+----------+-----------+--------------+----------------+--------------+--------------+---------+
|                nome|              nome_b|            mae|               mae_b| data_nasc|data_nasc_b|tipo_sanguineo|tipo_sanguineo_b|           cpf|         cpf_b|distancia|
+--------------------+--------------------+---------------+--------------------+----------+-----------+--------------+----------------+--------------+--------------+---------+
|Vicente Oliver Mo...|Felipe Bruno dos ...|Louise Beatriz |Isabelle Marcia A...|10/09/1948| 22/11/1965|           AB-|             AB-|097.627.958-49|506.865.202-46|     13.0|
|Vicente Oliver Mo...|Marcio Heitor Sil...|Louise Beatriz |       Alana Isadora|10/09/1948| 01/01/1980|           AB-|             AB-|097.627.958-49|325.828.867-42|     10.0|
|Vicente Oliver Mo...|Theo Mario Victor...|Louise Beatriz |  Caroline Gabriela |10/09/1948| 23/09/1961|           AB-|  

In [0]:
# Deduplicando a base (eliminando linhas duplicadas, ficando somente com a de maior similaridade)
dfc_5 = dfr_5.orderBy(['distancia'], ascending=False).dropDuplicates(['id_b']).orderBy(['id_b']).select(['id', 'id_b', 'nome','nome_b', 'mae', 'mae_b', 'cpf','cpf_b', 'distancia'])


In [0]:
# Criação de coluna de comparação de CPFs
dfc_5 = dfc_5.withColumn('CPFs_Iguais',F.col('cpf') == F.col('cpf_b'))

In [677]:
dfc_5.describe('distancia').show()

+-------+------------------+
|summary|         distancia|
+-------+------------------+
|  count|               143|
|   mean|10.244755244755245|
| stddev| 3.030866522348472|
|    min|              16.0|
|    max|               9.0|
+-------+------------------+



In [678]:
dfc_5.count()

143

In [679]:
dfc_5.filter(dfc_5['distancia'] >= '0.9').count()

143

In [680]:
dfc_5.filter(dfc_5['distancia'] >= '0.9').orderBy(['distancia'], ascending=False).show(150)

+---+----+--------------------+--------------------+--------------------+--------------------+--------------+--------------+---------+-----------+
| id|id_b|                nome|              nome_b|                 mae|               mae_b|           cpf|         cpf_b|distancia|CPFs_Iguais|
+---+----+--------------------+--------------------+--------------------+--------------------+--------------+--------------+---------+-----------+
| 12| 125|Matheus Claudio R...|   Samuel Noah Lopes|      Pietra Aurora |       Luana Louise |164.210.136-27|473.096.540-05|      9.0|      false|
| 10| 113|  Caio Kevin Pereira|        Claudio Rosa|     Mariana Helena |  Patricia Jennifer |999.008.988-42|748.193.267-08|      9.0|      false|
| 26|  34|   Samuel Noah Lopes|    Enzo Diogo Pinto|       Louise Joana |  Malu Sophia Marli |816.876.443-90|735.339.007-70|      9.0|      false|
|192| 133|Enrico Joaquim Ca...|Enrico Arthur Kau...|  Sarah Lucia Livia |  Maya Sara Isabela |076.678.965-93|348.748.0

In [0]:
dfc_5.limit(100).toPandas()

##Exportando Resultados para Verificação de Acurácia

In [699]:
dfc_4.select(F.col("cpf").alias("cpf_a"),'cpf_b', F.col("similaridade").alias("similarity")).show()

+--------------+--------------+------------------+
|         cpf_a|         cpf_b|        similarity|
+--------------+--------------+------------------+
|506.865.202-46|506.865.202-46|               1.0|
|325.828.867-42|325.828.867-42|0.9002136752136752|
|284.536.218-88|284.536.218-88|               1.0|
|390.375.925-20|390.375.925-20|0.9961538461538462|
|756.005.387-40|756.005.387-40|               1.0|
|551.341.139-39|551.341.139-39|0.9880952380952381|
|756.048.583-90|756.048.583-90|              0.75|
|386.351.591-95|386.351.591-95|0.7444444444444445|
|885.264.747-31|885.264.747-31|              0.75|
|489.611.821-96|489.611.821-96|0.7166666666666667|
|390.375.925-20|711.467.358-20|0.8101785714285714|
|919.624.227-20|823.068.573-85|0.8361111111111111|
|197.863.690-33|040.568.439-88|0.8547619047619047|
|853.220.078-81|461.427.999-69|0.8408730158730159|
|510.498.890-29|423.398.060-54|0.9380952380952381|
|621.824.822-45|974.659.231-90|              0.85|
|016.408.168-28|629.574.715-90|

In [0]:
dfc_4.select(F.col("cpf").alias("cpf_a"),'cpf_b', F.col("similaridade").alias("similarity")).toPandas().to_csv('gdrive/My Drive/Colab Notebooks/results_for_accuracy_calc.csv')

##Verificação de Acurácia

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
import pandas as pd

In [0]:
data = spark.read.csv('gdrive/My Drive/Colab Notebooks/results_for_accuracy_calc.csv', header=True)

In [706]:
data.limit(10).toPandas()

,_c0,cpf_a,cpf_b,similarity
0,0,506.865.202-46,506.865.202-46,1.0
1,1,325.828.867-42,325.828.867-42,0.9002136752136752
2,2,284.536.218-88,284.536.218-88,1.0
3,3,390.375.925-20,390.375.925-20,0.9961538461538462
4,4,756.005.387-40,756.005.387-40,1.0
5,5,551.341.139-39,551.341.139-39,0.9880952380952381
6,6,756.048.583-90,756.048.583-90,0.75
7,7,386.351.591-95,386.351.591-95,0.7444444444444445
8,8,885.264.747-31,885.264.747-31,0.75
9,9,489.611.821-96,489.611.821-96,0.7166666666666667


In [0]:
# Let us consider a cuttoff point set as 0.85
cutoff = 0.9

# sorting and deduplicating the resulting dataset
data = data.withColumn('similairty', F.col('similarity').cast(DoubleType()))
data = data.orderBy('similarity').dropDuplicates(['cpf_b'])
data = data.withColumn('match', F.when(F.col('similarity') >= cutoff, '1').otherwise('0'))

In [0]:
def inspect_pairs(cpf_a, cpf_b, match):
    if match == '1':
        if cpf_a == cpf_b:
            return "TP"
        else:
            return "FP"
    else:
        if cpf_a != cpf_b:
            return "TN"
        else: 
            return "FN"
udf_inspect_pairs = F.udf(inspect_pairs, StringType())

In [0]:
data = data.withColumn('perf', udf_inspect_pairs(F.col('cpf_a'), F.col('cpf_b'), F.col('match')))

## Measures
Accuracy = (TP + TN) / (FP + TP + FN + TN)

Positive predictive value (PPV) = TP / (TP + FP)

Negative predictive value (NPV) = TN / (TN + FN)

Sensitivity (True positive rate) = TP / (TP + FN)

Specificity (True negative rate) = TN / (TN + FP)

In [0]:
dic_results = {}
TP = data.filter(F.col('perf') == "TP").count()
TN = data.filter(F.col('perf') == "TN").count()
FP = data.filter(F.col('perf') == "FP").count()
FN = data.filter(F.col('perf') == "FN").count()


dic_results['accuracy'] =  float(TP + TN) / (FP + TP + FN + TN)
dic_results['ppv'] = float(TP) / (TP + FP)
dic_results['npv'] = float(TN) / (TN + FN)
dic_results['sens'] = float(TP) / (TP + FN)
dic_results['spec'] = float(TN) / (TN + FP)

In [0]:
final_results = pd.DataFrame(dic_results, index=[0])

In [712]:
final_results

,accuracy,npv,ppv,sens,spec
0,0.889831,0.784314,0.970149,0.855263,0.952381
